In [1]:
# 0.1. Cài đặt các thư viện cần thiết
!pip install -q fpdf2 noisereduce librosa tensorflow scikit-learn matplotlib seaborn pytz PyDrive2 


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# CÀI ĐẶT & CẤU HÌNH (SETUP & CONFIGURATION)
# 0.2. Import thư viện
import os
import glob 
import random
import datetime
import pytz
import shutil
import joblib
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from fpdf import FPDF
from tqdm import tqdm
import librosa
import noisereduce as nr
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from kaggle_secrets import UserSecretsClient
from oauth2client.service_account import ServiceAccountCredentials
from tensorflow.keras.regularizers import l2
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import roc_curve, auc as sklearn_auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
mixed_precision.set_global_policy('mixed_bfloat16')

try:
    # Cố gắng kết nối với TPU bằng cách chỉ định tpu='local'
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
    print('Đã tìm thấy TPU Resolver.')
    
    # Kết nối và khởi tạo hệ thống TPU
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    print('Đã khởi tạo hệ thống TPU.')

    # Tạo strategy
    strategy = tf.distribute.TPUStrategy(tpu)
    print('THÀNH CÔNG: Đã tạo TPUStrategy!')
    print(f'Số lượng nhân (replicas): {strategy.num_replicas_in_sync}')
    
except (ValueError, RuntimeError) as e:
    # Nếu vẫn không tìm thấy TPU, tự động chuyển về chiến lược mặc định
    print(f'Lỗi kết nối TPU: {e}')
    print('Không tìm thấy TPU. Sử dụng chiến lược mặc định cho GPU/CPU.')
    strategy = tf.distribute.get_strategy()

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0000 00:00:1756914650.644036      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230


Đã tìm thấy TPU Resolver.
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1756914667.876107      10 service.cc:148] XLA service 0x5d0b6608cbc0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756914667.876147      10 service.cc:156]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1756914667.876151      10 service.cc:156]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1756914667.876154      10 service.cc:156]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1756914667.876156      10 service.cc:156]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1756914667.876160      10 service.cc:156]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1756914667.876162      10 service.cc:156]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1756914667.876165      10 service.cc:156]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1756914667.876167      10 service.cc:156]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
Đã khởi tạo hệ thống TPU.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/

In [3]:
# THIẾT LẬP CẤU HÌNH 
# --- Các cấu hình cơ bản ---
SEED = 42
def set_seed(seed_value):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    #os.environ['TF_DETERMINISTIC_OPS'] = '1'
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
set_seed(SEED)

KAGGLE_PROCESSED_DATA_PATH = "/kaggle/input/ngt-spectrogram-id/"
KAGGLE_OUTPUT_PATH = "/kaggle/working/output_results"
os.makedirs(KAGGLE_OUTPUT_PATH, exist_ok=True)

CLASSES_TO_TRAIN = ['covid', 'asthma', 'healthy', 'tuberculosis']
ALL_CLASSES = ['healthy', 'asthma', 'covid', 'tuberculosis']
N_SPLITS = 5
TEST_SPLIT_RATIO = 0.15
USE_DATA_AUGMENTATION = True # Bật/tắt augmentation ở đây
USE_FOCAL_LOSS = True
USE_COSINE_DECAY_RESTARTS = True 

MODEL_ID = f'EfficienetB0_CV_TPU'
MIN_DELTA = 1e-4
SHUFFLE_BUFFER_SIZE = 2048 
GAMMA = 2.0 # Giữ nguyên giá trị tiêu chuẩn

# --- CÁC THAY ĐỔI CHÍNH ---
LEARNING_RATE = 5e-6              # Thay đổi: Giảm LR để ổn định hơn
WEIGHT_DECAY = 3e-3               # Thay đổi: Tăng để chống overfitting mạnh hơn

# Cấu hình cho Cross-Validation
TOTAL_EPOCHS = 200                # Giữ nguyên tổng số epochs tối đa
WARMUP_EPOCHS = 3                 # Thay đổi: Rút ngắn Warmup cho hiệu quả
RESTART_CYCLE_1_EPOCHS = 50       # Thay đổi: Chu kỳ ngắn hơn để khám phá nhiều hơn
PATIENCE_EPOCHS = RESTART_CYCLE_1_EPOCHS + 25 # Thay đổi: Patience = 50

# --- ĐỊNH NGHĨA BATCH SIZE ---
# BATCH_SIZE này là batch size cho mỗi nhân TPU (per-replica)
BATCH_SIZE = 32
# Tính toán GLOBAL_BATCH_SIZE để dùng trong pipeline
# Biến 'strategy' được lấy từ ô code đầu tiên
GLOBAL_BATCH_SIZE = BATCH_SIZE * strategy.num_replicas_in_sync
print(f"Batch size mỗi nhân: {BATCH_SIZE}")
print(f"Global batch size (tổng cộng): {GLOBAL_BATCH_SIZE}")

# INPUT_SHAPE sẽ được cập nhật lại ở ô chuẩn bị dữ liệu
INPUT_SHAPE = (224, 224, 3)

Batch size mỗi nhân: 32
Global batch size (tổng cộng): 256


In [4]:
# KHỞI TẠO CÁC HÀM CẦN THIẾT (PHIÊN BẢN ĐÚNG)

def get_patient_id(filepath, class_name):
    filename = os.path.basename(filepath)
    if class_name.lower() in ['asthma', 'covid', 'healthy']:
        return filename.split('_')[0]
    elif class_name.lower() == 'tuberculosis':
        return '_'.join(filename.split('_')[:-1]).replace('.npy', '')
    else:
        return filename.split('_')[0]

@register_keras_serializable()
class MacroF1Score(tf.keras.metrics.Metric):
    """
    Lớp metric để tính toán Macro F1-Score một cách chính xác trên toàn bộ epoch.
    """
    def __init__(self, num_classes, name='f1_macro', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.true_positives = self.add_weight(name='tp', shape=(num_classes,), initializer='zeros')
        self.false_positives = self.add_weight(name='fp', shape=(num_classes,), initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', shape=(num_classes,), initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_labels = tf.argmax(tf.nn.softmax(y_pred), axis=1)
        y_true_labels = tf.argmax(y_true, axis=1)
        cm = tf.math.confusion_matrix(y_true_labels, y_pred_labels, num_classes=self.num_classes, dtype=tf.float32)
        tp = tf.linalg.diag_part(cm)
        fp = tf.reduce_sum(cm, axis=0) - tp
        fn = tf.reduce_sum(cm, axis=1) - tp
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        macro_f1 = tf.reduce_mean(f1)
        return macro_f1

    def reset_state(self):
        self.true_positives.assign(tf.zeros(self.num_classes))
        self.false_positives.assign(tf.zeros(self.num_classes))
        self.false_negatives.assign(tf.zeros(self.num_classes))

    # Thêm phương thức get_config
    def get_config(self):
        config = super(MacroF1Score, self).get_config()
        config.update({'num_classes': self.num_classes})
        return config

    def reset_state(self):
        # Reset các biến trạng thái về 0 ở đầu mỗi epoch
        self.true_positives.assign(tf.zeros(self.num_classes))
        self.false_positives.assign(tf.zeros(self.num_classes))
        self.false_negatives.assign(tf.zeros(self.num_classes))
def parse_tfrecord_fn(example):
    """Hàm đọc và xử lý một mẫu từ file TFRecord cho EfficientNet."""
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    image = tf.io.parse_tensor(example['image'], out_type=tf.float32)
    image = tf.reshape(image, (256, 126))
    
    # --- THAY ĐỔI QUAN TRỌNG ---
    # 1. Stack 3 kênh TRƯỚC KHI resize để tạo thành ảnh 3D
    image_3d = tf.stack([image, image, image], axis=-1)
    
    # 2. Bây giờ mới resize ảnh 3D này về kích thước mong muốn
    image_resized = tf.image.resize(image_3d, [224, 224])
    
    # 3. Sử dụng hàm tiền xử lý chuyên dụng của EfficientNet
    image_preprocessed = preprocess_input(image_resized)
    
    label_encoded = tf.cast(example['label'], tf.int32)
    label_onehot = tf.one_hot(label_encoded, depth=len(ALL_CLASSES))
    
    return image_preprocessed, label_onehot

def augment(spectrogram, label):
    spectrogram = spec_augment(spectrogram)
    return spectrogram, label

def focal_loss_from_logits_optimized(alpha, gamma=2.0):
    """
    Tạo ra hàm Focal Loss phiên bản đầy đủ và sạch sẽ.
    
    Args:
        alpha: Một list hoặc array chứa trọng số cho mỗi lớp.
        gamma: Hệ số tập trung, mặc định là 2.0.
    """
    # Chuyển alpha sang dạng tensor để tính toán
    alpha = tf.constant(alpha, dtype=tf.float32)

    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, 'float32')
        y_pred = tf.cast(y_pred, 'float32')
        
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
        probs = tf.nn.softmax(y_pred)
        pt = tf.reduce_sum(y_true * probs, axis=-1)
        focal_term = (1.0 - pt) ** gamma
        alpha_t = tf.reduce_sum(y_true * alpha, axis=-1)
        loss = alpha_t * focal_term * cross_entropy
        
        return tf.reduce_mean(loss)
        
    return focal_loss_fixed

def spec_augment(spectrogram, time_masking_para=50, frequency_masking_para=40, num_time_masks=1, num_freq_masks=1):
    spectrogram_aug = spectrogram
    freq_bins = tf.shape(spectrogram)[1] # Sửa: Lấy chiều tần số từ shape 4D
    time_steps = tf.shape(spectrogram)[2] # Sửa: Lấy chiều thời gian từ shape 4D
    
    for _ in range(num_freq_masks):
        f = tf.random.uniform(shape=(), minval=0, maxval=frequency_masking_para, dtype=tf.int32)
        f0 = tf.random.uniform(shape=(), minval=0, maxval=freq_bins - f, dtype=tf.int32)
        freq_mask_1d = tf.concat([tf.ones((f0,), dtype=spectrogram.dtype), tf.zeros((f,), dtype=spectrogram.dtype), tf.ones((freq_bins - f0 - f,), dtype=spectrogram.dtype)], axis=0)
        freq_mask_4d = tf.reshape(freq_mask_1d, (1, freq_bins, 1, 1)) # Sửa: Reshape thành 4D để broadcast
        spectrogram_aug *= freq_mask_4d
        
    for _ in range(num_time_masks):
        t = tf.random.uniform(shape=(), minval=0, maxval=time_masking_para, dtype=tf.int32)
        t0 = tf.random.uniform(shape=(), minval=0, maxval=time_steps - t, dtype=tf.int32)
        time_mask_1d = tf.concat([tf.ones((t0,), dtype=spectrogram.dtype), tf.zeros((t,), dtype=spectrogram.dtype), tf.ones((time_steps - t0 - t,), dtype=spectrogram.dtype)], axis=0)
        time_mask_4d = tf.reshape(time_mask_1d, (1, 1, time_steps, 1)) # Sửa: Reshape thành 4D để broadcast
        spectrogram_aug *= time_mask_4d
        
    return spectrogram_aug

# HÀM CREATE_MODEL PHIÊN BẢN ĐÚNG VÀ ĐƠN GIẢN
@register_keras_serializable()
class FinalModel(tf.keras.Model):
    def __init__(self, input_shape, num_classes, **kwargs):
        super(FinalModel, self).__init__(**kwargs)
        self.input_shape_config = input_shape
        self.num_classes_config = num_classes
        
        self.base_model = EfficientNetB0(
            weights='imagenet',
            include_top=False,
            input_shape=self.input_shape_config
        )
        
        self.pooling = GlobalAveragePooling2D()
        self.dense1 = Dense(256, activation='relu', kernel_regularizer=l2(0.001)) # Lớp mới
        self.dropout = Dropout(0.5)
        self.dense_output = Dense(num_classes, activation='linear', dtype='float32')

    def call(self, inputs, training=None):
        x = self.pooling(x)
        x = self.dense1(x) # Thêm vào
        x = self.dropout(x, training=training)
        outputs = self.dense_output(x)
        return outputs

    # Thêm phương thức get_config
    def get_config(self):
        # Lấy config của lớp cha
        config = super(FinalModel, self).get_config()
        # Cập nhật với các tham số của lớp con
        config.update({
            'input_shape': self.input_shape_config,
            'num_classes': self.num_classes_config
        })
        return config

def load_data_from_df(df):
    X, y = [], []
    for _, row in df.iterrows():
        X.append(np.load(row['filepath']))
        y.append(row['label'])
    return np.array(X), np.array(y)

def get_grad_cam_final(model, img_array, last_conv_layer_name, pred_index=None):
    """
    Tạo Grad-CAM cho một subclassed model.
    Lưu ý: Model phải được build (chạy qua dữ liệu một lần) trước khi gọi hàm này.
    """
    # Tạo một model trung gian với input là input của model chính,
    # và output là lớp conv cuối và output cuối cùng của model chính.
    grad_model = Model(
        inputs=model.inputs,
        outputs=[model.base_model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Tính toán gradient
    with tf.GradientTape() as tape:
        # Đưa ảnh vào grad_model để lấy 2 output đã định nghĩa ở trên
        last_conv_layer_output, preds = grad_model(img_array)
        
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Lấy gradient của lớp được dự đoán đối với feature map của lớp conv cuối
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Tính trung bình gradient và tạo heatmap
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + tf.keras.backend.epsilon())
    
    return heatmap.numpy()

def overlay_grad_cam(spec, heatmap, alpha=0.6):
    heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], (spec.shape[0], spec.shape[1]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    jet = plt.cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap_resized.squeeze()]
    spec_display = np.stack([spec]*3, axis=-1)
    spec_display = (spec_display - spec_display.min()) / (spec_display.max() - spec_display.min())
    superimposed_img = jet_heatmap * alpha + spec_display
    superimposed_img = np.clip(superimposed_img, 0, 1)
    return superimposed_img

class PDFReport(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'BAO CAO KET QUA HUAN LUYEN MO HINH AI', 0, 1, 'C')
        self.ln(10)
    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Trang {self.page_no()}', 0, 0, 'C')
    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)
    def chapter_body(self, content):
        self.set_font('Arial', '', 10)
        safe_content = content.encode('latin-1', 'replace').decode('latin-1')
        self.multi_cell(0, 5, safe_content)
        self.ln()
    def add_image_section(self, title, img_path):
        self.chapter_title(title)
        if os.path.exists(img_path):
            self.image(img_path, x=None, y=None, w=180)
            self.ln(5)
        else:
            self.chapter_body(f"Khong tim thay hinh anh: {img_path}")

def authenticate_gdrive():
    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("google_service_account_key")
    with open("service_account.json", "w") as f:
        f.write(secret_value)
    scope = ["https://www.googleapis.com/auth/drive"]
    gauth = GoogleAuth()
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    drive = GoogleDrive(gauth)
    return drive

def upload_folder_to_drive(drive, folder_path, parent_folder_id):
    folder_name = os.path.basename(folder_path)
    print(f"Đang tạo thư mục '{folder_name}' trên Google Drive...")
    folder_metadata = {'title': folder_name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [{'id': parent_folder_id}]}
    folder = drive.CreateFile(folder_metadata)
    folder.Upload()
    
    print(f"Bắt đầu tải nội dung của '{folder_name}'...")
    for item in tqdm(os.listdir(folder_path), desc=f"Uploading {folder_name}"):
        item_path = os.path.join(folder_path, item)
        if os.path.isfile(item_path):
            gfile = drive.CreateFile({'title': item, 'parents': [{'id': folder['id']}]})
            gfile.SetContentFile(item_path)
            gfile.Upload(param={'supportsTeamDrives': True})
        elif os.path.isdir(item_path):
            upload_folder_to_drive(drive, item_path, folder['id'])

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(image, label):
    """Creates a tf.train.Example message ready to be written to a file."""
    feature = {
        'image': _bytes_feature(tf.io.serialize_tensor(image)),
        'label': _int64_feature(label)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        current_lr = tf.keras.backend.get_value(self.model.optimizer.learning_rate)
        print(f"\nEpoch {epoch+1}: Learning Rate is {current_lr:.2e}")

In [5]:
# CHUẨN BỊ DỮ LIỆU VÀ TẠO TFRECORD
suspicious_files_to_remove = [
    '/kaggle/input/ngt-spectrogram-id/healthy/P0030101_123370_Dkwg3F7jMGaR7kbc-seg2.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0032202_15897_PcbyJQWemBfghUYp-seg2.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0027142_5701_hupBI5CxKMNCfe8b-seg1.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0056214_89533_0WsmNRSKuQFGodg1-seg1.npy',
]
# --- BƯỚC 1: TẢI VÀ PHÂN CHIA DỮ LIỆU BAN ĐẦU ---
print("Bắt đầu chuẩn bị và phân chia dữ liệu...")
all_files_to_split = []
for class_name in ALL_CLASSES:
    source_dir = os.path.join(KAGGLE_PROCESSED_DATA_PATH, class_name)
    if os.path.exists(source_dir):
        files = glob.glob(os.path.join(source_dir, '*.npy'))
        for f in files:
            all_files_to_split.append({'filepath': f, 'label': class_name})

all_data_df = pd.DataFrame(all_files_to_split)
all_data_df['patient_id'] = all_data_df.apply(lambda row: get_patient_id(row['filepath'], row['label']), axis=1)

print(f"Số lượng mẫu ban đầu: {len(all_data_df)}")
all_data_df = all_data_df[~all_data_df['filepath'].isin(suspicious_files_to_remove)].reset_index(drop=True)
print(f"Số lượng mẫu sau khi lọc bỏ file 'im lặng': {len(all_data_df)}")

print("Tách tập Test cuối cùng (Hold-out set)...")
patient_ids = all_data_df['patient_id'].unique()
np.random.shuffle(patient_ids)
test_patient_count = int(len(patient_ids) * TEST_SPLIT_RATIO)
test_patients = patient_ids[:test_patient_count]
train_val_patients = patient_ids[test_patient_count:]

test_df = all_data_df[all_data_df['patient_id'].isin(test_patients)].reset_index(drop=True)
train_val_df = all_data_df[all_data_df['patient_id'].isin(train_val_patients)].reset_index(drop=True)

print(f"Đã tách: {len(train_val_df)} mẫu cho Train/Validation (CV) và {len(test_df)} mẫu cho Test cuối cùng.")

# --- BƯỚC 2: KHỞI TẠO LABEL ENCODER VÀ STANDARD SCALER ---
le = LabelEncoder().fit(ALL_CLASSES)
# Cập nhật INPUT_SHAPE từ một file mẫu
sample_spec = np.load(train_val_df['filepath'][0])
print(f"Kích thước input được cập nhật: {INPUT_SHAPE}")


# --- BƯỚC 3: CHUYỂN ĐỔI DỮ LIỆU SANG ĐỊNH DẠNG TFRECORD ---
TFRECORD_OUTPUT_PATH = "/kaggle/working/tfrecords"
os.makedirs(TFRECORD_OUTPUT_PATH, exist_ok=True)
print(f"Bắt đầu chuyển đổi dữ liệu sang TFRecord tại: {TFRECORD_OUTPUT_PATH}")

all_dfs = {'train_val': train_val_df, 'test': test_df}

for df_name, df in all_dfs.items():
    print(f"--- Đang xử lý tập {df_name} ---")
    tfrecord_path = os.path.join(TFRECORD_OUTPUT_PATH, f"{df_name}.tfrec")
    
    with tf.io.TFRecordWriter(tfrecord_path) as writer:
        # Sử dụng tqdm tiêu chuẩn để tránh lỗi ImportError
        for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Creating {df_name}.tfrec"):
            spectrogram = np.load(row['filepath']).astype(np.float32)
            label_encoded = le.transform([row['label']])[0]
            
            example = serialize_example(spectrogram, label_encoded)
            writer.write(example)
            
print("\nChuyển đổi sang TFRecord hoàn tất!")

Bắt đầu chuẩn bị và phân chia dữ liệu...
Số lượng mẫu ban đầu: 33084
Số lượng mẫu sau khi lọc bỏ file 'im lặng': 33080
Tách tập Test cuối cùng (Hold-out set)...
Đã tách: 28060 mẫu cho Train/Validation (CV) và 5020 mẫu cho Test cuối cùng.
Kích thước input được cập nhật: (224, 224, 3)
Bắt đầu chuyển đổi dữ liệu sang TFRecord tại: /kaggle/working/tfrecords
--- Đang xử lý tập train_val ---


Creating train_val.tfrec: 100%|██████████| 28060/28060 [04:58<00:00, 93.92it/s] 


--- Đang xử lý tập test ---


Creating test.tfrec: 100%|██████████| 5020/5020 [00:57<00:00, 87.27it/s] 



Chuyển đổi sang TFRecord hoàn tất!


In [6]:
# HUẤN LUYỆN MÔ HÌNH Cross-Validation

# --- BƯỚC 1: KHỞI TẠO CÁC BIẾN CẦN THIẾT ---
print("Đang khởi tạo các biến cho Cross-Validation...")
AUTOTUNE = tf.data.AUTOTUNE
skf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
cv_data_to_split = train_val_df[train_val_df['label'].isin(CLASSES_TO_TRAIN)]
X_cv_paths = cv_data_to_split['filepath'].values
y_cv_labels = le.transform(cv_data_to_split['label'])
groups_cv = cv_data_to_split['patient_id'].values

fold_accuracies, fold_losses, fold_aucs, fold_f1s = [], [], [], []

print("Đang tính toán trọng số alpha cho Focal Loss...")
class_weights_array = class_weight.compute_class_weight('balanced', classes=np.unique(y_cv_labels), y=y_cv_labels)
alpha_weights_list = class_weights_array.tolist()
print("Trọng số Alpha được tính toán:")
for i, w in enumerate(alpha_weights_list):
    class_name = le.inverse_transform([i])[0]
    print(f"- Lớp '{class_name}': {w:.2f}")

# --- BƯỚC 2: BẮT ĐẦU VÒNG LẶP CROSS-VALIDATION ---
LOCAL_TFRECORD_PATH = TFRECORD_OUTPUT_PATH
TRAIN_VAL_TFREC = os.path.join(LOCAL_TFRECORD_PATH, 'train_val.tfrec')
print(f"Sẵn sàng đọc dữ liệu TFRecord từ: {TRAIN_VAL_TFREC}")

for fold, (train_indices, val_indices) in enumerate(skf.split(X_cv_paths, y_cv_labels, groups_cv)):
    fold_number = fold + 1
    print("-" * 50 + f"\nBắt đầu Fold {fold_number}/{N_SPLITS}\n" + "-" * 50)
    # TÍNH TOÁN CÁC BƯỚC CHO SCHEDULER
    # 1. Tạo pipeline dữ liệu trước
    train_indices_tf = tf.constant(train_indices, dtype=tf.int64)
    val_indices_tf = tf.constant(val_indices, dtype=tf.int64)
    full_ds = tf.data.TFRecordDataset(TRAIN_VAL_TFREC).enumerate()
    train_ds = full_ds.filter(lambda i, data: tf.reduce_any(i == train_indices_tf)).map(lambda i, data: data)
    val_ds = full_ds.filter(lambda i, data: tf.reduce_any(i == val_indices_tf)).map(lambda i, data: data)
    train_ds = train_ds.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE).repeat().batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    
    # 2. Bây giờ mới tính toán steps_per_epoch
    steps_per_epoch = len(train_indices) // GLOBAL_BATCH_SIZE
    validation_steps = len(val_indices) // GLOBAL_BATCH_SIZE
    print(f"Số bước mỗi epoch: {steps_per_epoch} | Số bước validation: {validation_steps}")


    # --- TẠO MODEL VÀ LOSS FUNCTION ---
    with strategy.scope():
        model = FinalModel(input_shape=INPUT_SHAPE, num_classes=len(ALL_CLASSES))
        if USE_FOCAL_LOSS:
            loss_function = focal_loss_from_logits_optimized(alpha=alpha_weights_list, gamma=GAMMA)
        else:
            loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # --- GIAI ĐOẠN 1: HUẤN LUYỆN CÁC LỚP CUỐI (HEAD TRAINING) ---
    print("\n--- Bắt đầu Giai đoạn 1: Huấn luyện các lớp cuối ---")
    with strategy.scope():
        # Đóng băng toàn bộ mô hình nền
        model.base_model.trainable = False
        # Compile với learning rate lớn hơn (ví dụ 1e-3)
        optimizer_head = tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=WEIGHT_DECAY)
        model.compile(optimizer=optimizer_head, loss=loss_function, metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
        # Huấn luyện trong 5 epochs
        model.fit(train_ds, validation_data=val_ds, epochs=5,
              steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, verbose=1)

    # --- GIAI ĐOẠN 2: WARMUP VÀ HUẤN LUYỆN CHÍNH VỚI RESTART ---
    print("\n--- Bắt đầu Giai đoạn 2: Fine-tuning ---")

    # 1. THIẾT LẬP CÁC SIÊU THAM SỐ MỚI CHO LỊCH TRÌNH
    with strategy.scope():
        # Đếm tổng số lớp trong mô hình nền (EfficientNetB0)
        num_base_layers = len(model.base_model.layers)
        # Tính toán chỉ số của lớp bắt đầu mở băng (từ giữa mô hình trở đi)
        fine_tune_at = int(num_base_layers * 0.5) # 50%
        # Đầu tiên, đặt toàn bộ mô hình nền là không thể huấn luyện
        model.base_model.trainable = False
        # Bây giờ, mở băng các lớp từ chỉ số `fine_tune_at` đến cuối
        for layer in model.base_model.layers[fine_tune_at:]:
            layer.trainable = True
    
        print(f"Tổng số lớp trong base model: {num_base_layers}")
        print(f"Sẽ mở băng và huấn luyện {num_base_layers - fine_tune_at} lớp cuối cùng (bắt đầu từ lớp {fine_tune_at}).")

        f1_macro = MacroF1Score(num_classes=len(ALL_CLASSES), name='f1_macro')

        # 2. GIAI ĐOẠN 2A: WARMUP
        print(f"\n--- Giai đoạn 2A: Bắt đầu Warmup trong {WARMUP_EPOCHS} epochs ---")
        warmup_lr = LEARNING_RATE / 10 # Bắt đầu với LR rất thấp
        optimizer_warmup = tf.keras.optimizers.AdamW(learning_rate=warmup_lr, weight_decay=WEIGHT_DECAY)
        model.compile(optimizer=optimizer_warmup, loss=loss_function, metrics=['accuracy', f1_macro])

        model.fit(
            train_ds, validation_data=val_ds, epochs=WARMUP_EPOCHS,
            steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, verbose=1
        )

        # 3. GIAI ĐOẠN 2B: HUẤN LUYỆN CHÍNH VỚI COSINE DECAY RESTARTS
        print(f"\n--- Giai đoạn 2B: Bắt đầu huấn luyện chính ---")

        # Sử dụng công tắc để chọn optimizer và callbacks
        if USE_COSINE_DECAY_RESTARTS:
            print("Sử dụng scheduler: CosineDecayRestarts")
            first_decay_steps = RESTART_CYCLE_1_EPOCHS * steps_per_epoch
            lr_scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(
                initial_learning_rate=LEARNING_RATE,
                first_decay_steps=first_decay_steps,
                t_mul=2.0, 
                m_mul=0.9  # Giảm biên độ restart để ổn định hơn
            )
            optimizer_finetune = tf.keras.optimizers.AdamW(learning_rate=lr_scheduler, weight_decay=WEIGHT_DECAY)
            
            # Callbacks cho CosineDecayRestarts
            callbacks = [
                EarlyStopping(
                    monitor='val_f1_macro', mode='max', patience=PATIENCE_EPOCHS,
                    restore_best_weights=True, min_delta=MIN_DELTA, verbose=1
                ),
                LearningRateLogger()
            ]
        else:
            print("Sử dụng scheduler: ReduceLROnPlateau")
            # Optimizer với learning rate ban đầu cố định
            optimizer_finetune = tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
            
            # Callbacks cho ReduceLROnPlateau
            callbacks = [
                EarlyStopping(
                    monitor='val_f1_macro', mode='max', patience=30, # Có thể cần patience dài hơn
                    restore_best_weights=True, min_delta=MIN_DELTA, verbose=1
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_f1_macro', mode='max', factor=0.2,
                    patience=10, # Giảm LR nếu F1 không cải thiện trong 10 epochs
                    min_lr=1e-7,
                    verbose=1
                ),
                LearningRateLogger()
            ]

        # Biên dịch lại mô hình với optimizer và metrics cuối cùng
        with strategy.scope():
            model.compile(optimizer=optimizer_finetune, loss=loss_function, metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), f1_macro])
        
        # Huấn luyện mô hình
        history = model.fit(
            train_ds, validation_data=val_ds, epochs=TOTAL_EPOCHS,
            initial_epoch=WARMUP_EPOCHS,
            steps_per_epoch=steps_per_epoch, validation_steps=validation_steps,
            callbacks=callbacks, verbose=1
        )

    # Tạo đường dẫn và tên file để lưu model
    model_save_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
    # Lưu lại toàn bộ mô hình (kiến trúc + trọng số)
    model.save(model_save_path)
    # In ra thông báo để xác nhận
    print(f"Đã lưu mô hình cho Fold {fold_number} tại: {model_save_path}")

    # --- VẼ BIỂU ĐỒ VÀ ĐÁNH GIÁ ---
    print("Đang tạo và lưu biểu đồ huấn luyện...")
    plt.figure(figsize=(18, 7))
    plt.suptitle(f'Training Metrics for Fold {fold_number}', fontsize=16)
    
    # --- Biểu đồ cho các chỉ số (Accuracy, AUC, F1-Macro) ---
    plt.subplot(1, 2, 1)
    # Vẽ các chỉ số của tập Train
    plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linestyle='-')
    plt.plot(history.history['auc'], label='Training AUC', color='green', linestyle='-')
    plt.plot(history.history['f1_macro'], label='Training F1-Macro', color='red', linestyle='-')
    
    # Vẽ các chỉ số của tập Validation
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='blue', linestyle='--')
    plt.plot(history.history['val_auc'], label='Validation AUC', color='green', linestyle='--')
    plt.plot(history.history['val_f1_macro'], label='Validation F1-Macro', color='red', linestyle='--')
    
    # Cập nhật lại tiêu đề và nhãn
    plt.title('Biểu đồ các chỉ số (Accuracy, AUC, F1-Macro)')
    plt.xlabel('Epoch')
    plt.ylabel('Giá trị')
    plt.legend(loc='lower right')
    plt.grid(True)
    
    # --- Biểu đồ cho Loss ---
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss', color='orange')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='purple')
    plt.title('Biểu đồ Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)
    
    # Lưu và đóng hình ảnh
    plot_filename = f'fold_{fold_number}_metrics.png'
    plot_filepath = os.path.join(KAGGLE_OUTPUT_PATH, plot_filename)
    plt.savefig(plot_filepath)
    plt.close()
    
    print(f"Đã lưu biểu đồ cho Fold {fold_number} tại: {plot_filepath}")
    
    # THÊM 4: Sửa lại model.evaluate để nhận đủ 4 giá trị
    loss, accuracy, auc, f1 = model.evaluate(val_ds, verbose=0)
    print(f"Fold {fold_number} - Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}, Validation AUC: {auc:.4f}, Validation F1-Macro: {f1:.4f}")
    
    fold_aucs.append(auc)
    fold_losses.append(loss)
    fold_accuracies.append(accuracy)
    fold_f1s.append(f1) # Thêm lưu trữ F1

    # THÊM 5: Sửa lại câu lệnh print cuối cùng
    print("=" * 50 + "\nKết quả Cross-Validation:\n" 
      + f"Validation Accuracy trung bình: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}\n"
      + f"Validation Loss trung bình: {np.mean(fold_losses):.4f} +/- {np.std(fold_losses):.4f}\n"
      + f"Validation AUC trung bình: {np.mean(fold_aucs):.4f} +/- {np.std(fold_aucs):.4f}\n"
      + f"Validation F1-Macro trung bình: {np.mean(fold_f1s):.4f} +/- {np.std(fold_f1s):.4f}\n" # Thêm dòng này
      + "=" * 50)

Đang khởi tạo các biến cho Cross-Validation...
Đang tính toán trọng số alpha cho Focal Loss...
Trọng số Alpha được tính toán:
- Lớp 'asthma': 1.82
- Lớp 'covid': 0.80
- Lớp 'healthy': 0.87
- Lớp 'tuberculosis': 0.96
Sẵn sàng đọc dữ liệu TFRecord từ: /kaggle/working/tfrecords/train_val.tfrec
--------------------------------------------------
Bắt đầu Fold 1/5
--------------------------------------------------
Cause: could not parse the source code of <function <lambda> at 0x7fd0d45a7760>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
lambda i, data: data

Match 1:
lambda i, data: tf.reduce_any(i == train_indices_tf)

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7fd0d45a7760>: found multiple definitions with identic

I0000 00:00:1756900668.298453      10 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

--- Bắt đầu Giai đoạn 1: Huấn luyện các lớp cuối ---
Epoch 1/5


I0000 00:00:1756900699.170761      10 encapsulate_tpu_computations_pass.cc:266] Subgraph fingerprint:10934389281517148678
I0000 00:00:1756900701.834812     947 tpu_compilation_cache_interface.cc:442] TPU host compilation cache miss: cache_key(2486306964948227037), session_name()
I0000 00:00:1756900725.458849     947 tpu_compile_op_common.cc:245] Compilation of 2486306964948227037 with session name  took 23.623338785s and succeeded
I0000 00:00:1756900725.529632     947 tpu_compilation_cache_interface.cc:476] TPU host compilation cache: compilation complete for cache_key(2486306964948227037), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_10934389281517148678", property.function_library_fingerprint = 7262044275807281730, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().

87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.6322 - auc: 0.7673 - loss: 6.8659

...

87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.4124 - auc: 0.6216 - f1_macro: 0.2088 - loss: 6.2783

I0000 00:00:1756910736.529493      10 encapsulate_tpu_computations_pass.cc:266] Subgraph fingerprint:8491900130754903393
I0000 00:00:1756910737.765494     901 tpu_compilation_cache_interface.cc:442] TPU host compilation cache miss: cache_key(9251591330270504796), session_name()
I0000 00:00:1756910750.839646     901 tpu_compile_op_common.cc:245] Compilation of 9251591330270504796 with session name  took 13.07377707s and succeeded
I0000 00:00:1756910750.871468     901 tpu_compilation_cache_interface.cc:476] TPU host compilation cache: compilation complete for cache_key(9251591330270504796), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_8491900130754903393", property.function_library_fingerprint = 554851425115163362, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = 

87/87 ━━━━━━━━━━━━━━━━━━━━ 100s 428ms/step - accuracy: 0.4106 - auc: 0.6202 - f1_macro: 0.2090 - loss: 6.3022 - val_accuracy: 0.4018 - val_auc: 0.6182 - val_f1_macro: 0.3576 - val_loss: 5.7184

Epoch 5: Learning Rate is 5.00e-06
Epoch 5/200
87/87 ━━━━━━━━━━━━━━━━━━━━ 16s 185ms/step - accuracy: 0.3706 - auc: 0.6101 - f1_macro: 0.2246 - loss: 6.1609 - val_accuracy: 0.4183 - val_auc: 0.6570 - val_f1_macro: 0.3911 - val_loss: 5.4105

Epoch 6: Learning Rate is 4.98e-06
Epoch 6/200
87/87 ━━━━━━━━━━━━━━━━━━━━ 15s 179ms/step - accuracy: 0.3768 - auc: 0.6001 - f1_macro: 0.2745 - loss: 6.0784 - val_accuracy: 0.4319 - val_auc: 0.6678 - val_f1_macro: 0.4007 - val_loss: 5.3030

Epoch 7: Learning Rate is 4.96e-06
Epoch 7/200
87/87 ━━━━━━━━━━━━━━━━━━━━ 17s 199ms/step - accuracy: 0.3760 - auc: 0.6000 - f1_macro: 0.3086 - loss: 5.9803 - val_accuracy: 0.4403 - val_auc: 0.6741 - val_f1_macro: 0.4003 - val_loss: 5.2324

Epoch 8: Learning Rate is 4.92e-06
Epoch 8/200
87/87 ━━━━━━━━━━━━━━━━━━━━ 17s 199ms/st

I0000 00:00:1756912907.510011      10 encapsulate_tpu_computations_pass.cc:266] Subgraph fingerprint:17842064229865454063
I0000 00:00:1756912908.801210     913 tpu_compilation_cache_interface.cc:442] TPU host compilation cache miss: cache_key(10503501604871670389), session_name()
I0000 00:00:1756912921.816997     913 tpu_compile_op_common.cc:245] Compilation of 10503501604871670389 with session name  took 13.015713702s and succeeded
I0000 00:00:1756912921.836606     913 tpu_compilation_cache_interface.cc:476] TPU host compilation cache: compilation complete for cache_key(10503501604871670389), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_17842064229865454063", property.function_library_fingerprint = 14351898955023293915, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wra

Fold 5 - Validation Loss: 3.9951, Validation Accuracy: 0.5266, Validation AUC: 0.7365, Validation F1-Macro: 0.4803
Kết quả Cross-Validation:
Validation Accuracy trung bình: 0.5269 +/- 0.0076
Validation Loss trung bình: 4.1705 +/- 0.1343
Validation AUC trung bình: 0.7454 +/- 0.0099
Validation F1-Macro trung bình: 0.4726 +/- 0.0149


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tqdm.notebook import tqdm

print("Bắt đầu quy trình Knowledge Distillation...")

# --- BƯỚC 1: TẠO KIẾN THỨC TỪ ENSEMBLE 5 MÔ HÌNH THẦY GIÁO ---

print("Bước 1: Tạo các nhãn mềm từ 5 mô hình thầy giáo...")

# 1.1. Tạo lại dataset huấn luyện (không shuffle, không repeat) để lấy dự đoán
train_val_ds_for_preds = tf.data.TFRecordDataset(TRAIN_VAL_TFREC)
train_val_ds_for_preds = train_val_ds_for_preds.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
train_val_ds_for_preds = train_val_ds_for_preds.batch(GLOBAL_BATCH_SIZE)
train_val_ds_for_preds = train_val_ds_for_preds.prefetch(buffer_size=AUTOTUNE)

# 1.2. Lặp qua 5 folds để lấy và cộng dồn logits
all_fold_logits = []
for fold_number in range(1, N_SPLITS + 1):
    print(f"  - Đang lấy dự đoán từ Fold {fold_number}...")
    model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
    with strategy.scope():
        teacher_model = tf.keras.models.load_model(model_path)
    
    fold_logits = teacher_model.predict(train_val_ds_for_preds, verbose=0)
    all_fold_logits.append(fold_logits)

# 1.3. Tính trung bình logits và tạo nhãn mềm (xác suất)
mean_logits = np.mean(np.stack(all_fold_logits, axis=0), axis=0)
soft_labels = tf.nn.softmax(mean_logits).numpy()

# 1.4. Lấy nhãn thật (hard labels)
hard_labels_onehot = np.concatenate([label.numpy() for _, label in train_val_ds_for_preds], axis=0)

# 1.5. Lấy dữ liệu ảnh (X_train)
X_train_spectrograms = np.concatenate([img.numpy() for img, _ in train_val_ds_for_preds], axis=0)

print(f"-> Đã tạo xong {len(soft_labels)} nhãn mềm.")


# --- BƯỚC 2: XÂY DỰNG MÔ HÌNH HỌC SINH ---

def create_student_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Sử dụng mixed precision cho mô hình học sinh để tăng tốc
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, (3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(3, strides=2)(x)

    x = layers.Conv2D(128, (3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(3, strides=2)(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, dtype='float32')(x) # Output logits
    return keras.Model(inputs, outputs)

print("Bước 2: Đã định nghĩa kiến trúc mô hình học sinh.")


# --- BƯỚC 3: TẠO LỚP DISTILLER ---

class Distiller(keras.Model):
    def __init__(self, student):
        super().__init__()
        self.student = student

    def compile(self, optimizer, metrics, student_loss_fn, distillation_loss_fn, alpha=0.1, temperature=3):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data
        teacher_predictions = y["soft"]
        true_labels = y["hard"]

        with tf.GradientTape() as tape:
            # Lấy dự đoán từ mô hình học sinh (logits)
            student_predictions = self.student(x, training=True)

            # Tính student loss với nhãn thật
            student_loss = self.student_loss_fn(true_labels, student_predictions)

            # Tính distillation loss với nhãn mềm của thầy
            # Dùng temperature để làm "mềm" hơn nữa distribition
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )

            # Kết hợp 2 loss lại
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Cập nhật trọng số
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Cập nhật metrics
        self.compiled_metrics.update_state(true_labels, student_predictions)
        
        # Trả về kết quả
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss, "distillation_loss": distillation_loss})
        return results

print("Bước 3: Đã định nghĩa lớp Distiller.")


# --- BƯỚC 4: HUẤN LUYỆN MÔ HÌNH HỌC SINH ---
print("Bước 4: Bắt đầu huấn luyện mô hình học sinh...")

# Tạo dataset cuối cùng cho việc huấn luyện
# Dữ liệu X là spectrogram, dữ liệu y là một dictionary chứa cả 2 loại nhãn
distillation_ds = tf.data.Dataset.from_tensor_slices(
    (X_train_spectrograms, {"soft": soft_labels, "hard": hard_labels_onehot})
).batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)


with strategy.scope():
    # Khởi tạo mô hình học sinh
    student = create_student_model(input_shape=INPUT_SHAPE, num_classes=len(ALL_CLASSES))
    
    # Khởi tạo Distiller
    distiller = Distiller(student=student)
    
    # Compile Distiller
    distiller.compile(
        optimizer=keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4),
        metrics=[keras.metrics.CategoricalAccuracy(name="accuracy")],
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
        distillation_loss_fn=keras.losses.KLDivergence(),
        alpha=0.2, # Trọng số cho hard loss, 80% học từ thầy
        temperature=5, # Làm mềm distribition
    )

# Huấn luyện
distiller.fit(distillation_ds, epochs=50) # Tăng epochs nếu cần

# Lưu lại mô hình học sinh sau khi huấn luyện
student_model_path = os.path.join(KAGGLE_OUTPUT_PATH, "student_model.keras")
distiller.student.save(student_model_path)
print(f"Đã huấn luyện và lưu mô hình học sinh tại: {student_model_path}")

In [9]:
# ĐÁNH GIÁ SO SÁNH 5-FOLD, ENSEMBLE VÀ MÔ HÌNH HỌC SINH

print("Bắt đầu quy trình đánh giá toàn diện...")

# --- BƯỚC 1: CHUẨN BỊ DỮ LIỆU TEST (CHỈ LÀM MỘT LẦN) ---
TEST_TFREC = os.path.join(TFRECORD_OUTPUT_PATH, 'test.tfrec')
print(f"Sử dụng dữ liệu test từ: {TEST_TFREC}")

test_ds = tf.data.TFRecordDataset(TEST_TFREC)
test_ds = test_ds.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(GLOBAL_BATCH_SIZE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

y_test_onehot = np.concatenate([label.numpy() for _, label in test_ds], axis=0)
y_test_encoded = np.argmax(y_test_onehot, axis=1)
target_names = le.classes_
n_classes = len(target_names)
run_timestamp = datetime.datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).strftime("%Y%m%d_%H%M")
report_figs_path = os.path.join(KAGGLE_OUTPUT_PATH, "final_report_figures")
os.makedirs(report_figs_path, exist_ok=True)

# ==============================================================================
# PHẦN 1: ĐÁNH GIÁ ENSEMBLE TỪ 5 FOLDS
# ==============================================================================
print("\n" + "="*30 + "\nPHẦN 1: ĐÁNH GIÁ ENSEMBLE\n" + "="*30)

all_fold_preds_logits = []
for fold_number in range(1, N_SPLITS + 1):
    print(f"  - Lấy dự đoán từ Fold {fold_number}...")
    model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
    with strategy.scope():
        teacher_model = tf.keras.models.load_model(model_path)
    all_fold_preds_logits.append(teacher_model.predict(test_ds, verbose=0))

mean_logits = np.mean(np.stack(all_fold_preds_logits, axis=0), axis=0)
y_pred_probs_ensemble = tf.nn.softmax(mean_logits).numpy()
y_pred_encoded_ensemble = np.argmax(y_pred_probs_ensemble, axis=1)

print("\nBáo cáo chi tiết của mô hình Ensemble:")
print(classification_report(y_test_encoded, y_pred_encoded_ensemble, target_names=target_names))

# Vẽ Ma trận nhầm lẫn cho Ensemble
cm_ensemble = confusion_matrix(y_test_encoded, y_pred_encoded_ensemble)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_ensemble, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.title(f'Ma trận nhầm lẫn (Ensemble trên {N_SPLITS} Folds)', fontsize=16)
plt.ylabel('Nhãn thật'); plt.xlabel('Nhãn dự đoán')
cm_ensemble_path = os.path.join(report_figs_path, f'ensemble_confusion_matrix_{run_timestamp}.png')
plt.savefig(cm_ensemble_path)
plt.show(); plt.close()

# (Các biểu đồ so sánh 5-fold và ROC chi tiết có thể giữ lại hoặc bỏ đi tuỳ bạn)


# ==============================================================================
# PHẦN 2: ĐÁNH GIÁ MÔ HÌNH HỌC SINH
# ==============================================================================
print("\n" + "="*30 + "\nPHẦN 2: ĐÁNH GIÁ MÔ HÌNH HỌC SINH\n" + "="*30)

student_model_path = os.path.join(KAGGLE_OUTPUT_PATH, "student_model.keras")

if not os.path.exists(student_model_path):
    print(f"Không tìm thấy mô hình học sinh tại '{student_model_path}'. Bỏ qua bước đánh giá này.")
else:
    print("Đang tải mô hình học sinh...")
    with strategy.scope():
        student_model = tf.keras.models.load_model(student_model_path)
        # Biên dịch lại model để có thể tính toán đầy đủ các metrics
        student_model.compile(
            loss=keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), MacroF1Score(num_classes=n_classes)]
        )

    print("Đang đánh giá mô hình học sinh trên tập test...")
    student_loss, student_acc, student_auc, student_f1 = student_model.evaluate(test_ds, verbose=1)
    
    print("Đang dự đoán với mô hình học sinh...")
    y_pred_probs_student = tf.nn.softmax(student_model.predict(test_ds, verbose=0)).numpy()
    y_pred_encoded_student = np.argmax(y_pred_probs_student, axis=1)

    # --- Lưu kết quả ra file text ---
    student_report_text = classification_report(y_test_encoded, y_pred_encoded_student, target_names=target_names)
    results_filepath = os.path.join(KAGGLE_OUTPUT_PATH, "student_model_evaluation_results.txt")
    with open(results_filepath, "w") as f:
        f.write("--- KET QUA DANH GIA MO HINH HOC SINH ---\n\n")
        f.write(f"Timestamp: {run_timestamp}\n\n")
        f.write(f"Test Loss: {student_loss:.4f}\n")
        f.write(f"Test Accuracy: {student_acc:.4f}\n")
        f.write(f"Test AUC: {student_auc:.4f}\n")
        f.write(f"Test F1-Macro: {student_f1:.4f}\n")
        f.write("\n--- BAO CAO PHAN LOAI CHI TIET ---\n")
        f.write(student_report_text)
    print(f"\nĐã lưu kết quả đánh giá của mô hình học sinh vào file: {results_filepath}")
    print(student_report_text)

    # --- Vẽ biểu đồ riêng cho mô hình học sinh ---
    # 1. Ma trận nhầm lẫn (Học sinh)
    cm_student = confusion_matrix(y_test_encoded, y_pred_encoded_student)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_student, annot=True, fmt='d', cmap='Greens', xticklabels=target_names, yticklabels=target_names)
    plt.title('Ma trận nhầm lẫn (Mô hình Học sinh)', fontsize=16)
    plt.ylabel('Nhãn thật'); plt.xlabel('Nhãn dự đoán')
    cm_student_path = os.path.join(report_figs_path, f'student_confusion_matrix_{run_timestamp}.png')
    plt.savefig(cm_student_path)
    plt.show(); plt.close()

    # 2. Đường cong ROC (Học sinh)
    y_test_binarized = label_binarize(y_test_encoded, classes=range(n_classes))
    plt.figure(figsize=(12, 10))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_probs_student[:, i])
        auc_score = sklearn_auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'Lớp: {target_names[i]} (AUC = {auc_score:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Chance')
    plt.title('Đường cong ROC (Mô hình Học sinh)', fontsize=16)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right'); plt.grid(True)
    roc_student_path = os.path.join(report_figs_path, f'student_roc_curves_{run_timestamp}.png')
    plt.savefig(roc_student_path)
    plt.show(); plt.close()

    print(f"\nĐã hoàn tất đánh giá mô hình học sinh! Các biểu đồ được lưu tại: {report_figs_path}")


Bắt đầu đánh giá so sánh 5 mô hình từ Cross-Validation trên tập Test...
Sử dụng dữ liệu test từ: /kaggle/working/tfrecords/test.tfrec
--------------------------------------------------
Đang xử lý Fold 1/5


TypeError: Could not locate class 'FinalModel'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'FinalModel', 'config': {'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'mixed_bfloat16'}, 'registered_name': None}}, 'registered_name': 'FinalModel', 'build_config': {'input_shape': [32, 224, 224, 3]}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'AdamW', 'config': {'name': 'adamw', 'learning_rate': {'module': 'keras.optimizers.schedules', 'class_name': 'CosineDecayRestarts', 'config': {'initial_learning_rate': 5e-06, 'first_decay_steps': 4350, 't_mul': 2.0, 'm_mul': 0.9, 'alpha': 0.0, 'name': 'SGDRDecay'}, 'registered_name': None}, 'weight_decay': 0.003, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gra

In [ ]:
# VẼ GRAD-CAM CHO MÔ HÌNH ENSEMBLE (PHIÊN BẢN SỬA LỖI HOÀN CHỈNH)

# --- BƯỚC 0: TẢI 5 MODELS VÀ LẤY DỰ ĐOÁN ENSEMBLE ---
# (Giả định rằng các biến từ ô đánh giá trước đó vẫn tồn tại, ví dụ: test_ds, y_test_encoded)

all_fold_preds_logits = []
for fold_number in range(1, N_SPLITS + 1):
    print(f"Đang tải dự đoán từ Fold {fold_number}...")
    model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
    with strategy.scope():
        model = tf.keras.models.load_model(model_path) # Không cần custom_objects vì đã đăng ký
    preds_logits = model.predict(test_ds, verbose=0)
    all_fold_preds_logits.append(preds_logits)

# Lấy dự đoán cuối cùng bằng cách trung bình hóa logits
mean_logits = np.mean(np.stack(all_fold_preds_logits, axis=0), axis=0)
y_pred_probs_ensemble = tf.nn.softmax(mean_logits).numpy()
y_pred_encoded_ensemble = np.argmax(y_pred_probs_ensemble, axis=1)

print("Đã có dự đoán từ mô hình ensemble.")


# --- BƯỚC 1: LẤY DỮ LIỆU TEST TỪ PIPELINE `test_ds` (NẾU CẦN CHẠY LẠI) ---
print("Đang trích xuất dữ liệu ảnh từ test_ds để vẽ Grad-CAM...")
X_test_list = [images.numpy() for images, _ in test_ds]
X_test = np.concatenate(X_test_list, axis=0)
print(f"Đã trích xuất thành công {len(X_test)} mẫu ảnh.")


# --- BƯỚC 2: CHỈ ĐỊNH LỚP CONVOLUTION CUỐI CÙNG ---
# Gán trực tiếp tên lớp, không cần tìm kiếm tự động
last_conv_layer_name = "top_conv" 
print(f"Sử dụng lớp Grad-CAM được chỉ định: {last_conv_layer_name}")


# --- BƯỚC 3: TẠO HÌNH ẢNH GRAD-CAM CHO CÁC MẪU TIÊU BIỂU (DÙNG KẾT QUẢ ENSEMBLE) ---
# Tải lại một mô hình bất kỳ để làm sườn cho việc tính Grad-CAM
# Vì kiến trúc giống nhau, ta có thể dùng bất kỳ model nào từ 5 folds
print("Tải một mô hình mẫu để tính Grad-CAM...")
with strategy.scope():
    sample_model_for_gradcam = tf.keras.models.load_model(os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_1.keras'))

run_timestamp = datetime.datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).strftime("%Y%m%d_%H%M")
gradcam_path = os.path.join(KAGGLE_OUTPUT_PATH, "grad_cam_figures")
os.makedirs(gradcam_path, exist_ok=True)
print("Tạo hình ảnh Grad-CAM cho các mẫu tiêu biểu...")

results_list = []
for i in range(len(y_test_encoded)):
    is_correct = (y_test_encoded[i] == y_pred_encoded_ensemble[i])
    results_list.append({
        'index': i, 
        'true_label': y_test_encoded[i], 
        'pred_label': y_pred_encoded_ensemble[i], 
        'confidence': y_pred_probs_ensemble[i][y_pred_encoded_ensemble[i]], 
        'is_correct': is_correct
    })
results_df = pd.DataFrame(results_list)

# Lấy tên các lớp
target_names = le.classes_
trained_class_indices = range(len(target_names))

for class_index, class_name in enumerate(target_names):
    correct_samples = results_df[(results_df['is_correct'] == True) & (results_df['true_label'] == class_index)].nlargest(3, 'confidence')
    incorrect_samples = results_df[(results_df['is_correct'] == False) & (results_df['true_label'] == class_index)].nlargest(3, 'confidence')
    
    for _, row in correct_samples.iterrows():
        idx = int(row['index'])
        img_array, spec = X_test[idx][np.newaxis, ...], X_test[idx, :, :, 0]
        heatmap = get_grad_cam_final(sample_model_for_gradcam, img_array, last_conv_layer_name, pred_index=class_index)
        overlay = overlay_grad_cam(spec, heatmap)
        plt.imshow(overlay)
        plt.title(f"Dự đoán Đúng: {class_name}\n(Tin cậy: {row['confidence']:.2f})")
        plt.axis('off')
        plt.savefig(os.path.join(gradcam_path, f"correct_{class_name}_{idx}_{run_timestamp}.png"))
        plt.close()
        
    for _, row in incorrect_samples.iterrows():
        idx = int(row['index'])
        pred_class_name = target_names[int(row['pred_label'])]
        img_array, spec = X_test[idx][np.newaxis, ...], X_test[idx, :, :, 0]
        # Tính heatmap theo nhãn thật để xem tại sao mô hình nhầm
        heatmap = get_grad_cam_final(sample_model_for_gradcam, img_array, last_conv_layer_name, pred_index=int(row['true_label']))
        overlay = overlay_grad_cam(spec, heatmap)
        plt.imshow(overlay)
        plt.title(f"Thật: {class_name}, Sai -> {pred_class_name}\n(Tin cậy: {row['confidence']:.2f})")
        plt.axis('off')
        plt.savefig(os.path.join(gradcam_path, f"incorrect_{class_name}_as_{pred_class_name}_{idx}_{run_timestamp}.png"))
        plt.close()

print("Hoàn tất việc tạo Grad-CAM.")

In [ ]:
# TẠO BÁO CÁO PDF TỔNG HỢP 5-FOLD VÀ ENSEMBLE

print("Bắt đầu tạo báo cáo PDF tổng hợp...")

# --- BƯỚC 1: TRÍCH XUẤT CÁC KẾT QUẢ CẦN THIẾT ---

# Lấy kết quả trung bình từ validation set (từ ô huấn luyện)
val_acc_mean = np.mean(fold_accuracies)
val_acc_std = np.std(fold_accuracies)
val_loss_mean = np.mean(fold_losses)
val_loss_std = np.std(fold_losses)
val_f1_mean = np.mean(fold_f1s)
val_f1_std = np.std(fold_f1s)

# Lấy kết quả trung bình từ test set (từ ô đánh giá)
test_metrics_mean = metrics_df.mean()

# Lấy báo cáo chi tiết của mô hình ensemble
ensemble_report_text = classification_report(y_test_encoded, y_pred_encoded_ensemble, target_names=target_names)
ensemble_report_dict = classification_report(y_test_encoded, y_pred_encoded_ensemble, target_names=target_names, output_dict=True)
ensemble_accuracy = ensemble_report_dict['accuracy']
ensemble_f1_macro = ensemble_report_dict['macro avg']['f1-score']


# --- BƯỚC 2: TẠO FILE PDF ---
pdf = PDFReport()
pdf.add_page()

# --- Phần 1: Tóm tắt cấu hình và kết quả ---
pdf.chapter_title("1. Tom tat cau hinh va Ket qua")
# Lấy timestamp từ ô đánh giá để thống nhất
run_timestamp = datetime.datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).strftime("%Y-%m-%d %H:%M:%S")

config_summary = f"""
- Model ID: {MODEL_ID}
- Thoi gian chay: {run_timestamp}
- Cac lop huan luyen: {', '.join(CLASSES_TO_TRAIN)}
- K-Fold Cross-Validation: {N_SPLITS} folds

--- KET QUA CROSS-VALIDATION (TRUNG BINH TREN TAP VALIDATION) ---
- Validation Accuracy: {val_acc_mean:.4f} +/- {val_acc_std:.4f}
- Validation F1-Macro: {val_f1_mean:.4f} +/- {val_f1_std:.4f}
- Validation Loss: {val_loss_mean:.4f} +/- {val_loss_std:.4f}

--- KET QUA ENSEMBLE (TREN TAP TEST CUOI CUNG) ---
- Ensemble Test Accuracy: {ensemble_accuracy:.4f}
- Ensemble Test F1-Macro: {ensemble_f1_macro:.4f}

--- CAU HINH CHI TIET ---
- SEED: {SEED}
- Tong so Epochs toi da: {TOTAL_EPOCHS} (Patience: {PATIENCE_EPOCHS})
- Batch Size: {BATCH_SIZE} (Global: {GLOBAL_BATCH_SIZE})
- Learning Rate ban dau: {LEARNING_RATE}
- Ham Loss: {'Focal Loss' if USE_FOCAL_LOSS else 'Categorical Crossentropy'}
- Tang cuong du lieu: {'Co' if USE_DATA_AUGMENTATION else 'Khong'}
- Kich thuoc Input: {INPUT_SHAPE}
"""
pdf.chapter_body(config_summary)


# --- Phần 2: Đánh giá chi tiết trên tập Test ---
pdf.add_page()
pdf.chapter_title("2. Danh gia chi tiet tren tap Test")
pdf.chapter_body("Bao cao phan loai chi tiet cua mo hinh Ensemble:")
pdf.set_font('Courier', '', 8)
pdf.chapter_body(ensemble_report_text)
pdf.set_font('Arial', '', 10) # Reset font

# Thêm các biểu đồ so sánh đã tạo
pdf.add_image_section("So sanh Ma tran nham lan cua 5 Folds:", cm_plot_path)
pdf.add_image_section("So sanh Duong cong ROC cua 5 Folds (theo tung lop):", roc_plot_path)


# --- Phần 3: Phân tích Grad-CAM ---
pdf.add_page()
pdf.chapter_title("3. Phan tich Grad-CAM (dua tren du doan Ensemble)")

# Lấy timestamp của lần chạy Grad-CAM gần nhất để tìm đúng file ảnh
gradcam_run_timestamp = os.path.basename(cm_plot_path).replace('comparison_confusion_matrix_', '').replace('.png', '')

for class_name in target_names:
    pdf.chapter_body(f"Lop: {class_name}")
    
    # Tìm các ảnh Grad-CAM tương ứng
    correct_imgs = sorted(glob.glob(os.path.join(gradcam_path, f"correct_{class_name}_*_{gradcam_run_timestamp}.png")))
    incorrect_imgs = sorted(glob.glob(os.path.join(gradcam_path, f"incorrect_{class_name}_as_*_{gradcam_run_timestamp}.png")))
    
    x_pos, y_pos = pdf.get_x(), pdf.get_y()
    
    if correct_imgs:
        pdf.chapter_body("Vi du du doan dung:")
        y_pos = pdf.get_y()
        for i, img_path in enumerate(correct_imgs[:3]):
            if os.path.exists(img_path): pdf.image(img_path, x=x_pos + i * 60, y=y_pos, w=55)
        pdf.ln(45) # Xuống dòng
    
    if incorrect_imgs:
        pdf.chapter_body("Vi du du doan sai:")
        y_pos = pdf.get_y()
        for i, img_path in enumerate(incorrect_imgs[:3]):
            if os.path.exists(img_path): pdf.image(img_path, x=x_pos + i * 60, y=y_pos, w=55)
        pdf.ln(45) # Xuống dòng

# --- BƯỚC 3: LƯU FILE PDF VÀ TẢI LÊN DRIVE ---
report_filename = f"report_5fold_{MODEL_ID}_{run_timestamp}.pdf"
report_filepath = os.path.join(KAGGLE_OUTPUT_PATH, report_filename)
pdf.output(report_filepath)
print(f"Da tao bao cao PDF tai: {report_filepath}")

# Phần tải lên Google Drive (giữ nguyên)
# print("\nBắt đầu quá trình tải kết quả lên Google Drive...")
# drive = authenticate_gdrive()
# upload_folder_to_drive(drive, KAGGLE_OUTPUT_PATH, 'YOUR_DRIVE_FOLDER_ID') # Thay ID folder của bạn vào đây
# print("Hoàn tất! Toàn bộ kết quả đã được lưu về Google Drive.")